<a href="https://colab.research.google.com/github/tusharvatsa32/Language-Models/blob/main/Bag_of_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install dynet

     |████████████████████████████████| 4.5MB 11.8MB/s 


In [3]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [6]:
!git clone https://github.com/tusharvatsa32/nn4nlp-code.git

Cloning into 'nn4nlp-code'...
remote: Enumerating objects: 545, done.
remote: Total 545 (delta 0), reused 0 (delta 0), pack-reused 545
Receiving objects: 100% (545/545), 11.92 MiB | 3.49 MiB/s, done.
Resolving deltas: 100% (217/217), done.


In [42]:
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])


In [43]:
train = list(read_dataset("nn4nlp-code/data/classes/train.txt"))
print(train[0])
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("nn4nlp-code/data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)
print(nwords,ntags)


([1, 2, 3, 4, 5, 6, 1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 9, 17, 5, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], 0)
18648 5


In [44]:
print(train[0])
print(train[1])
print(train[2])
print(len(train[0][0]),len(train[1][0]),len(train[2][0]))

([1, 2, 3, 4, 5, 6, 1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 9, 17, 5, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], 0)
([1, 34, 35, 36, 37, 11, 1, 38, 37, 1, 39, 13, 40, 3, 41, 42, 15, 19, 43, 37, 44, 45, 46, 47, 48, 49, 50, 51, 9, 52, 53, 37, 54, 55, 9, 56, 33], 1)
([57, 58, 59, 60, 19, 61, 37, 62, 63, 19, 64, 65, 66, 26, 19, 64, 67, 68, 69, 5, 1, 70, 63, 71, 1, 72, 73, 74, 75, 1, 76, 26, 77, 26, 78, 37, 1, 79, 33], 0)
36 37 39


In [45]:
# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [46]:
W_sm = model.add_lookup_parameters((nwords, ntags)) # Word weights
b_sm = model.add_parameters((ntags))                # Softmax bias

In [47]:
def calc_scores(words):
  dy.renew_cg()
  score = dy.esum([dy.lookup(W_sm, x) for x in words])
  b_sm_exp = dy.parameter(b_sm)
  return score + b_sm_exp


In [48]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.5153, time=0.28s
iter 0: test acc=0.3643
iter 1: train loss/sent=1.3391, time=0.26s
iter 1: test acc=0.3729
iter 2: train loss/sent=1.2359, time=0.26s
iter 2: test acc=0.3896
iter 3: train loss/sent=1.1567, time=0.27s
iter 3: test acc=0.3982
iter 4: train loss/sent=1.0924, time=0.30s
iter 4: test acc=0.4036
iter 5: train loss/sent=1.0373, time=0.25s
iter 5: test acc=0.4158
iter 6: train loss/sent=0.9907, time=0.28s
iter 6: test acc=0.4059
iter 7: train loss/sent=0.9488, time=0.27s
iter 7: test acc=0.4100
iter 8: train loss/sent=0.9129, time=0.27s
iter 8: test acc=0.4095
iter 9: train loss/sent=0.8799, time=0.27s
iter 9: test acc=0.4217
iter 10: train loss/sent=0.8494, time=0.27s
iter 10: test acc=0.4222
iter 11: train loss/sent=0.8225, time=0.28s
iter 11: test acc=0.4186
iter 12: train loss/sent=0.7978, time=0.26s
iter 12: test acc=0.4072
iter 13: train loss/sent=0.7745, time=0.27s
iter 13: test acc=0.4163
iter 14: train loss/sent=0.7536, time=0.30s
iter 14: t